In [1]:
import torch
import numpy as np

In [2]:
import pandas as pd

In [3]:
from thesislib.utils.dl.models import DNN, DEFAULT_LAYER_CONFIG

In [4]:
model = DNN(383, 801, DEFAULT_LAYER_CONFIG)

In [5]:
state_dict_path = "/home/oagba/bulk/dl_runs/working/jupyter_basic_15k/run_good.torch"

In [6]:
state_dict = torch.load(state_dict_path)
model.load_state_dict(state_dict)

/shares/bulk/oagba/work/medvice-parser/lib64/python3.6/site-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla K40c with CUDA capability sm_35 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70 sm_75.
If you want to use the Tesla K40c GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


<All keys matched successfully>

In [7]:
age_mean = 38.741316816862515
age_std = 23.380120690086834

In [8]:
file = "/home/oagba/bulk/data/output_basic_15k/symptoms/csv/parsed/test.csv_sparse.csv"

In [9]:
df = pd.read_csv(file, index_col="Index")

/shares/bulk/oagba/work/medvice-parser/lib64/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
df.shape

(1129429, 5)

In [11]:
labels = df.LABEL
df = df.drop(columns=['LABEL'])

In [12]:
from thesislib.utils.dl.data import DLSparseMaker, AiBasicMedDataset

In [13]:
num_symptoms = 376
num_conditions = 801

In [14]:
sparsifier = DLSparseMaker(num_symptoms, age_mean, age_std)

In [15]:
data = sparsifier.transform(df)

In [16]:
dataset = AiBasicMedDataset(data, labels)

In [17]:
from torch.utils.data import DataLoader
from thesislib.utils.dl.utils import DeviceDataLoader, to_device, get_default_device

In [18]:
loader = DataLoader(
    dataset,
    batch_size=1024,
    shuffle=False,
    num_workers=4
)

In [19]:
device = get_default_device()

In [20]:
loader = DeviceDataLoader(loader,device)

In [21]:
import importlib

In [22]:
from thesislib.utils.dl import runners

In [23]:
importlib.reload(runners)

<module 'thesislib.utils.dl.runners' from '/shares/bulk/oagba/work/medvice-parser/lib64/python3.6/site-packages/thesislib/utils/dl/runners.py'>

In [24]:
# prec, acc, top5 = runners.calculate_precision_accuracy_top_5(model, loader, 801, None)

In [25]:
for item in loader:
    batch, batch_label = item
    break

In [27]:
batch_label.shape

torch.Size([1024])

In [28]:
out = model(batch)

In [29]:
out.shape

torch.Size([1024, 801])

In [30]:
_, y_pred = torch.max(out, dim=1)

In [34]:
y_pred = to_device(y_pred, device)

In [ ]:
torch.sum(y_pred == batch_label)

In [33]:
y_pred.is_cuda

False